In [1]:
import pandas as pd
import numpy as np

In [2]:
ls

AHP-Automation.ipynb          dataset.txt
Corrosion Rate - Vikor.ipynb  working-dataset.csv
dataset.csv.numbers*


In [11]:
data = pd.read_csv('working-dataset.csv')

In [12]:
data

,Material,Approx. Cost per Square Meter (USD),Durability (Months),Weight (kg/mÂ²),Corrosion Resistance,Temperature Resistance (Â°C),Manufacturing Complexity,Availability
0,Nickel-Based Superalloys,$750,10 years,300,Very High,1000,Medium,Very High
1,Stainless Steel,$200,8 years,200,High,250,Medium,High
2,Titanium Alloys,$1050,12 years,150,Very High,1100,High,Medium
3,Carbon Fiber Reinforced Polymers (CFRP),$800,5 years,50,Very High,300,Medium,Medium
4,Ceramic Matrix Composites (CMCs),$1350,12 years,50,Very High,1050,High,Low
5,Inconel,$700,15 years,200,Very High,800,Medium,Very High
6,Single Crystal Alloys,$1100,13 years,200,Very High,1100,High,Low


In [13]:
values_tomap = {
    'Very High':5,
    'High' :4,
    'Medium':3,
    'Low':2,
    'Very Low':1
}

In [14]:
# mapping five scale values
def mapping_five_scales(columns):
    for col in columns:
        data[col] = data[col].apply(lambda x:x.strip()).map(values_tomap)
    return data

In [15]:
# Use only if the columns name are irreggular
def regularization():
    for col in data.columns:
        data.rename(columns = {col : col[1:].strip()},inplace=True)

In [16]:
#regularization()

In [17]:
mapped_data = mapping_five_scales(['Corrosion Resistance','Manufacturing Complexity','Availability'])
mapped_data

,Material,Approx. Cost per Square Meter (USD),Durability (Months),Weight (kg/mÂ²),Corrosion Resistance,Temperature Resistance (Â°C),Manufacturing Complexity,Availability
0,Nickel-Based Superalloys,$750,10 years,300,5,1000,3,5
1,Stainless Steel,$200,8 years,200,4,250,3,4
2,Titanium Alloys,$1050,12 years,150,5,1100,4,3
3,Carbon Fiber Reinforced Polymers (CFRP),$800,5 years,50,5,300,3,3
4,Ceramic Matrix Composites (CMCs),$1350,12 years,50,5,1050,4,2
5,Inconel,$700,15 years,200,5,800,3,5
6,Single Crystal Alloys,$1100,13 years,200,5,1100,4,2


In [194]:
# generating pair-wise comparision based on user-preference
user_data = pd.read_csv('user-data.csv')

In [195]:
user_data

,Criteria,Cost,Durability,Weight,Corrosion Resistance,Temperature Resistance,Manufacturing Complexity,Availability
0,Cost,1,1/3,1/5,1/3,3,2,2
1,Durability,3,1,3,3,3,2,3
2,Weight,1/5,1/3,1,1/3,1/5,1/3,1/5
3,Corrosion Resistance,3,1/3,3,1,1,2,3
4,Temperature Resistance,1/3,1/3,5,1,1,2,2
5,Manufacturing Complexity,1/2,1/2,3,1/2,1/2,1,1
6,Availability,1/2,1/3,5,1/3,1/2,1,1


In [196]:
for col in user_data.columns:
    user_data.rename(columns = {col:col.strip()},inplace=True)

In [197]:
user_data.drop(columns=['Criteria'],inplace=True)

In [198]:
# calculating the float values
def division(x):
    try:
        if '/' in x:
            parts = x.split('/')
            return int(parts[0]) / int(parts[1])
        else:
            return int(x)
    except:
        return x
def float_values(df):
    try:
        df.drop(columns=['Pair-wise comp'],inplace=True)
    except:
        df = df
    for col in df.columns:
        df[col] = df[col].apply(lambda x:round(division(x),2))
    return df

In [199]:
def normalization(df):
    for col in df.columns:
        sum_ = round(df[col].sum(),2)
        df[col] = df[col].apply(lambda x:round(x/sum_,2))
    return df

In [200]:
def sum_of_rows(df):
    l=[]
    for i in range(df.shape[0]):
        l.append(round(sum(df.loc[i,:]),2))
    return l

In [201]:
def criterion_weight(df):
    l=[]
    for i in range(df.shape[0]):
        l.append(round(sum(df.loc[i,:])/df.shape[1],2))
    return l

In [202]:
def consistency_mat(caldf,l):
    for i,col in enumerate(caldf.columns):
        caldf[col] = caldf[col].apply(lambda x :l[i]*x)
    return caldf

In [203]:
def weighted_mat(df,cw):
    l = sum_of_rows(df)
    df['Weighted_sum_value'] = pd.Series(l)
    df['Criteria_weight'] = pd.Series(cw)
    df['result']  = df['Weighted_sum_value']/df['Criteria_weight']
    return df

In [204]:
cal_df = float_values(user_data)
cal_df

,Cost,Durability,Weight,Corrosion Resistance,Temperature Resistance,Manufacturing Complexity,Availability
0,1.00,0.33,0.2,0.33,3.0,2.00,2.0
1,3.00,1.00,3.0,3.00,3.0,2.00,3.0
2,0.20,0.33,1.0,0.33,0.2,0.33,0.2
3,3.00,0.33,3.0,1.00,1.0,2.00,3.0
4,0.33,0.33,5.0,1.00,1.0,2.00,2.0
5,0.50,0.50,3.0,0.50,0.5,1.00,1.0
6,0.50,0.33,5.0,0.33,0.5,1.00,1.0


In [205]:
cal_df_n = cal_df.copy() 
norm_df = normalization(cal_df)
norm_df

,Cost,Durability,Weight,Corrosion Resistance,Temperature Resistance,Manufacturing Complexity,Availability
0,0.12,0.10,0.01,0.05,0.33,0.19,0.16
1,0.35,0.32,0.15,0.46,0.33,0.19,0.25
2,0.02,0.10,0.05,0.05,0.02,0.03,0.02
3,0.35,0.10,0.15,0.15,0.11,0.19,0.25
4,0.04,0.10,0.25,0.15,0.11,0.19,0.16
5,0.06,0.16,0.15,0.08,0.05,0.10,0.08
6,0.06,0.10,0.25,0.05,0.05,0.10,0.08


In [206]:
cal_df_n

,Cost,Durability,Weight,Corrosion Resistance,Temperature Resistance,Manufacturing Complexity,Availability
0,1.00,0.33,0.2,0.33,3.0,2.00,2.0
1,3.00,1.00,3.0,3.00,3.0,2.00,3.0
2,0.20,0.33,1.0,0.33,0.2,0.33,0.2
3,3.00,0.33,3.0,1.00,1.0,2.00,3.0
4,0.33,0.33,5.0,1.00,1.0,2.00,2.0
5,0.50,0.50,3.0,0.50,0.5,1.00,1.0
6,0.50,0.33,5.0,0.33,0.5,1.00,1.0


In [207]:
cw = criterion_weight(norm_df)
cw

[0.14, 0.29, 0.04, 0.19, 0.14, 0.1, 0.1]

In [208]:
consistency_df = consistency_mat(cal_df_n,cw)
consistency_df

,Cost,Durability,Weight,Corrosion Resistance,Temperature Resistance,Manufacturing Complexity,Availability
0,0.1400,0.0957,0.008,0.0627,0.420,0.200,0.20
1,0.4200,0.2900,0.120,0.5700,0.420,0.200,0.30
2,0.0280,0.0957,0.040,0.0627,0.028,0.033,0.02
3,0.4200,0.0957,0.120,0.1900,0.140,0.200,0.30
4,0.0462,0.0957,0.200,0.1900,0.140,0.200,0.20
5,0.0700,0.1450,0.120,0.0950,0.070,0.100,0.10
6,0.0700,0.0957,0.200,0.0627,0.070,0.100,0.10


In [209]:
result_df = weighted_mat(consistency_df,cw)
result_df

,Cost,Durability,Weight,Corrosion Resistance,Temperature Resistance,Manufacturing Complexity,Availability,Weighted_sum_value,Criteria_weight,result
0,0.1400,0.0957,0.008,0.0627,0.420,0.200,0.20,1.13,0.14,8.071429
1,0.4200,0.2900,0.120,0.5700,0.420,0.200,0.30,2.32,0.29,8.000000
2,0.0280,0.0957,0.040,0.0627,0.028,0.033,0.02,0.31,0.04,7.750000
3,0.4200,0.0957,0.120,0.1900,0.140,0.200,0.30,1.47,0.19,7.736842
4,0.0462,0.0957,0.200,0.1900,0.140,0.200,0.20,1.07,0.14,7.642857
5,0.0700,0.1450,0.120,0.0950,0.070,0.100,0.10,0.70,0.10,7.000000
6,0.0700,0.0957,0.200,0.0627,0.070,0.100,0.10,0.70,0.10,7.000000


In [210]:
n=data.shape[1]-1
lam_max = np.array(result_df['result']).mean()
consistency_index = (lam_max - n)/(n-1)

In [211]:
lam_max

7.60016111707841

In [212]:
consistency_index

0.10002685284640167

In [213]:
consistent_Ratio = consistency_index/1.32

In [214]:
round(consistent_Ratio,2)

0.08